In [1]:
import requests
from config import DATA_DOWNLOAD_URL
import pandas as pd
import io
import gzip
import numpy as np
from datetime import datetime

In [2]:
df = pd.DataFrame(columns=["DATE", "DEP", "RR", "TN", "TX", "TM"])#, "UM"])

In [6]:
for i in range(95):
    print(i)
    url = f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_{i+1:02d}_previous-1950-2023_RR-T-Vent.csv.gz"
    response = requests.get(url)

    compressed_data = io.BytesIO(response.content)
    decompressed_data = gzip.GzipFile(fileobj=compressed_data)

    tmp = pd.read_csv(decompressed_data, sep=";")
    tmp["DATE"] = pd.to_datetime(tmp["AAAAMMJJ"], format="%Y%m%d") #string to date
    #tmp = tmp[tmp["DATE"] >= "1979-01-01"].reset_index(drop=True) #filter by date
    tmp['DEP'] = i+1
    tmp = tmp[["DATE", "DEP", "RR", "TN", "TX", "TM"]] #keep only, in order : Date, departement code, précipitations, temp min, temp max and moy temp
    df = pd.concat([df, tmp])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94


In [4]:
#data for current market year
for i in range(95):
    urlTP = f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_{i+1:02d}_previous-1950-2023_RR-T-Vent.csv.gz"
    responseTP = requests.get(urlTP)

    urlRH = f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_{i+1:02d}_previous-1950-2023_autres-parametres.csv.gz" 
    responseRH = requests.get(urlRH)

    compressedDataTP = io.BytesIO(responseTP.content)
    decompressedDataTP = gzip.GzipFile(fileobj=compressedDataTP)
    compressedDataRH = io.BytesIO(responseRH.content)
    decompressedDataRH = gzip.GzipFile(fileobj=compressedDataRH)

    tmpTP = pd.read_csv(decompressedDataTP, sep=";")
    tmpRH = pd.read_csv(decompressedDataRH, sep=";")
    tmp = pd.merge(tmpTP, tmpRH, on=["NUM_POSTE", "AAAAMMJJ"], how="outer")
    tmp["DATE"] = pd.to_datetime(tmp["AAAAMMJJ"], format="%Y%m%d") #string to date
    tmp['DEP'] = i+1
    tmp = tmp[["DATE", "DEP", "RR", "TN", "TX", "TM", "UM"]] #keep only, in order : Date, departement code, précipitations, temp min, temp max, moy temp and relative humidity
    df = pd.concat([df, tmp])
svpMin = 610.78 * np.exp(df["TN"] / (df["TN"] + 237.3) * 17.2694)
svpMean = 610.78 * np.exp(df["TM"] / (df["TM"] + 237.3) * 17.2694)
svpMax = 610.78 * np.exp(df["TX"] / (df["TX"] + 237.3) * 17.2694)
vpdMin = svpMin * (1 - df["UM"]/100) / 1000 #divided by 1000 to convert to kPa
vpdMean = svpMean * (1 - df["UM"]/100) / 1000
vpdMax = svpMax * (1 - df["UM"]/100) / 1000
df["vpd_min"] = vpdMin
df["vpd_mean"] = vpdMean
df["vpd_max"] = vpdMax

C:\Users\alexl\AppData\Local\Temp\ipykernel_15340\571040300.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, tmp])


In [4]:
df = df.dropna()
df['YEAR-MONTH'] = pd.to_datetime(df["DATE"]).dt.to_period('M') #new column with YYYY-MM format

In [7]:
df = df.reset_index(drop=True)

In [6]:
precipMeanOnEachDayForEachDep = df[["DATE", "DEP", "RR", "YEAR-MONTH"]].groupby(["DATE", "DEP"]).mean().reset_index()
precipSumForEachMonthForEachDep = precipMeanOnEachDayForEachDep[["DEP", "RR", "YEAR-MONTH"]].groupby(["DEP", "YEAR-MONTH"]).sum().reset_index()
tempMeanForEachMonthForEachDep = df[["DEP", "TN", "TX", "TM", "vpd_min", "vpd_mean", "vpd_max", "YEAR-MONTH"]].groupby(["DEP", "YEAR-MONTH"]).mean().reset_index()

weather = pd.merge(precipSumForEachMonthForEachDep, tempMeanForEachMonthForEachDep, on=["DEP", "YEAR-MONTH"], how="inner")

In [7]:
weather

,DEP,YEAR-MONTH,RR,TN,TX,TM,vpd_min,vpd_mean,vpd_max
0,1,1950-01,85.50,-1.273333,4.853333,1.680000,0.065089,0.082978,0.105477
1,1,1950-02,99.40,1.911538,10.780769,5.884615,0.165983,0.218654,0.314905
2,1,1950-03,29.40,0.853571,13.796429,7.150000,0.158750,0.248884,0.392141
3,1,1950-04,105.70,3.992000,13.776000,8.684000,0.188112,0.275093,0.410970
4,1,1950-05,114.10,9.745161,21.829032,15.893548,0.339302,0.509773,0.746062
...,...,...,...,...,...,...,...,...,...
74024,95,2023-08,81.74,14.638710,24.505161,19.236129,0.441457,0.600953,0.842193
74025,95,2023-09,67.00,14.377333,25.608000,19.627333,0.459400,0.666244,0.995983
74026,95,2023-10,81.78,10.125806,19.095484,14.224516,0.264827,0.361911,0.524128
74027,95,2023-11,100.20,6.228000,11.662000,8.671333,0.131855,0.152915,0.185879


In [9]:
df.to_csv(f"{DATA_DOWNLOAD_URL}/raw_rr_tn_tx_tm-1950-2023.csv")

In [6]:
df = df.reset_index(drop=True)

In [7]:
df.to_csv(f"{DATA_DOWNLOAD_URL}/rr_tn_tx_tm-current.csv")

In [ ]:
df.to_csv(f"{DATA_DOWNLOAD_URL}/rr_tn_tx_tm-1950-2023.csv")

Make a mean file for each month, each departement

In [3]:
df = pd.read_csv(f"{DATA_DOWNLOAD_URL}/rr_tn_tx_tm-1950-2023.csv")
df = df[df["DATE"] >= "2003-01-01"]
df = df.dropna()
df = df.drop(columns=["Unnamed: 0"])

In [4]:
dailyDepMean = df.groupby(["DEP", "DATE"]).mean().reset_index()
dailyDepMean['YEAR-MONTH'] = pd.to_datetime(dailyDepMean["DATE"]).dt.to_period('M') #new column with YYYY-MM format

In [5]:
yearlyPrecip = dailyDepMean.groupby(["DEP", "YEAR-MONTH"]).sum()["RR"].reset_index()
yearlyPrecip['MONTH'] = yearlyPrecip['YEAR-MONTH'].dt.month
monthlyPrecip = yearlyPrecip.groupby(["MONTH", "DEP"]).mean()['RR'].reset_index()

In [6]:
yearlyTemp = dailyDepMean[["DEP", "YEAR-MONTH", "TN", "TX", "TM"]].groupby(["DEP", "YEAR-MONTH"]).mean()[["TN", "TX", "TM"]].reset_index()
yearlyTemp['MONTH'] = yearlyTemp['YEAR-MONTH'].dt.month
monthlyTemp = yearlyTemp.groupby(["MONTH", "DEP"]).mean()[["TN", "TX", "TM"]].reset_index()

In [7]:
meteoFranceMean = pd.merge(monthlyPrecip, monthlyTemp, on=["MONTH", "DEP"], how='inner')
meteoFranceMean

,MONTH,DEP,RR,TN,TX,TM
0,1,1,111.142766,-1.012066,5.468150,1.859154
1,1,2,63.543750,1.204320,6.098548,3.519510
2,1,3,60.424884,0.512403,6.722931,3.370427
3,1,4,49.107585,-3.260030,8.497296,1.471727
4,1,5,53.761693,-4.779385,6.117791,-0.398876
...,...,...,...,...,...,...
1135,12,91,62.340454,2.318371,7.686998,4.862220
1136,12,92,46.580000,4.584866,9.176586,6.604274
1137,12,93,44.542857,4.257143,8.658894,6.340369
1138,12,94,64.388095,3.248771,8.242857,5.538479


In [8]:
meteoFranceMean.to_csv("../YieldModel/data/forecast/currentYear/temp_precip_mean_historical.csv")

Make the file for current year

In [86]:
df = pd.read_csv(f"{DATA_DOWNLOAD_URL}/rr_tn_tx_tm-current.csv")
df = df[pd.to_datetime(df["DATE"]) < datetime.today().replace(day=1, hour=0, minute=0, second=0, microsecond=0)] #remove data from current month
df = df.dropna()
df = df.drop(columns=["Unnamed: 0"])

In [88]:
dailyDepMean = df.groupby(["DEP", "DATE"]).mean().reset_index()
dailyDepMean['YEAR-MONTH'] = pd.to_datetime(dailyDepMean["DATE"]).dt.to_period('M') #new column with YYYY-MM format

In [89]:
yearlyPrecip = dailyDepMean.groupby(["DEP", "YEAR-MONTH"]).sum()["RR"].reset_index()
yearlyPrecip['MONTH'] = yearlyPrecip['YEAR-MONTH'].dt.month
monthlyPrecip = yearlyPrecip.groupby(["MONTH", "DEP"]).mean()['RR'].reset_index()

In [90]:
yearlyTemp = dailyDepMean[["DEP", "YEAR-MONTH", "TN", "TX", "TM", "vpd_min", "vpd_max", "vpd_mean"]].groupby(["DEP", "YEAR-MONTH"]).mean()[["TN", "TX", "TM", "vpd_min", "vpd_max", "vpd_mean"]].reset_index()
yearlyTemp['MONTH'] = yearlyTemp['YEAR-MONTH'].dt.month
monthlyTemp = yearlyTemp.groupby(["MONTH", "DEP"]).mean()[["TN", "TX", "TM", "vpd_min", "vpd_max", "vpd_mean"]].reset_index()

In [91]:
currentMeteoFranceMean = pd.merge(monthlyPrecip, monthlyTemp, on=["MONTH", "DEP"], how='inner')
currentMeteoFranceMean

,MONTH,DEP,RR,TN,TX,TM,vpd_min,vpd_max,vpd_mean
0,1,1,175.760000,-0.256129,7.592903,3.376129,0.105159,0.195878,0.142369
1,1,2,123.192857,0.885945,6.060753,3.212289,0.059307,0.087267,0.070670
2,1,3,68.440000,0.220000,8.271613,4.061290,0.126657,0.229078,0.171554
3,1,4,76.387500,-0.887097,10.548790,3.939113,0.124916,0.285799,0.177396
4,1,5,105.975000,-2.553495,7.302419,1.632527,0.122693,0.262170,0.169138
...,...,...,...,...,...,...,...,...,...
547,12,88,104.000000,-0.133692,5.143728,2.260573,0.054734,0.086540,0.067828
548,12,89,52.033333,2.449462,7.269892,4.776344,0.062145,0.088011,0.074244
549,12,90,100.500000,0.248387,5.509677,2.548387,0.058420,0.086967,0.069934
550,12,91,41.166667,3.551613,8.012903,5.716129,0.084960,0.113940,0.097801


In [93]:
currentMeteoFranceMean.to_csv("../YieldModel/data/forecast/currentYear/temp_precip_vpd_mean_current.csv")